# 🌱 Soil Classification Challenge - Inference
This notebook handles the inference pipeline for the Soil Classification Challenge. We load the trained model and label encoder, process test images, generate predictions, create a submission file, and visualize some test predictions.

## 1. 📚 Importing Required Libraries
We import the necessary libraries for image processing, model loading, and visualization.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder

## 2. 🏗️ Loading Model and Label Encoder
We load the trained CNN model and label encoder saved from the training phase.

In [ ]:
IMG_SIZE = 128

# Load the trained model
model = tf.keras.models.load_model('soil_classification_model.h5')

# Load the label encoder
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

print("✅ Model and label encoder loaded successfully.")
print("Label mapping:")
for i, soil_type in enumerate(label_encoder.classes_):
    print(f"{i}: {soil_type}")

## 3. 🧾 Test Data Prediction & Submission
We process test images, make predictions using the trained model, and create a submission CSV file.

In [ ]:
test_image_dir = "/content/soil_classification-2025/test"
test_images = []
image_ids = []

for img_name in tqdm(os.listdir(test_image_dir)):
    img_path = os.path.join(test_image_dir, img_name)
    try:
        image = Image.open(img_path).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
        image = np.array(image) / 255.0
        test_images.append(image)
        image_ids.append(img_name)
    except Exception as e:
        print(f"Error processing image {img_name}: {e}")

test_images = np.array(test_images)
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

submission_df = pd.DataFrame({
    'image_id': image_ids,
    'soil_type': predicted_labels
})
submission_df.to_csv('submission.csv', index=False)
print("✅ Submission file created successfully.")

## 4. 🖼️ Visualizing Test Predictions
Let's visualize some test images with their predicted soil types.

In [ ]:
plt.figure(figsize=(15, 10))
for i in range(min(9, len(test_images))):
    plt.subplot(3, 3, i+1)
    plt.imshow(test_images[i])
    plt.title(f"Pred: {predicted_labels[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## 5. ✅ Conclusion
- Successfully loaded the trained model and label encoder.
- Generated predictions for test data.
- Created a submission file and visualized test predictions.